----

Trabajas en una empresa emergente que vende productos alimenticios. Debes investigar el comportamiento del usuario para la aplicación de la empresa.

## Abrir el archivo de datos y leer la información general

In [ ]:
#Importar librerias
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import numpy as np

In [ ]:
#Creamos una ruta de archivo única para cargar el archivo
try:
    path = 'C:/Users/erika/Desktop/data/sprint_11/'
    df = pd.read_csv(path + 'logs_exp_us.csv', sep='\t')
except: 
    df = pd.read_csv('logs_exp_us.csv', sep='\t')
df.head()

## Preparar los datos para el análisis

### Cambia el nombre de las columnas de manera que sea conveniente para ti.

In [ ]:
df = df.rename(columns={"EventName": "name", "DeviceIDHash": "user_id", "EventTimestamp": "time", "ExpId": "exp_id"})
df.head()

### Comprueba si hay tipos de datos y valores ausentes. 

In [ ]:
df.info(show_counts=True)

Nos encontramos con que no hay datos ausentes.

In [ ]:
#Verificamos si hay duplicados
df.duplicated().sum()

In [ ]:
#visualizamos las filas duplicadas
duplicados = df[df.duplicated()]
duplicados.head(10)

Nos encontramos con duplicados de los tres grupos y de los diferentes eventos, por lo que podremos eliminarlos sin que afecten el análisis.

In [7]:
df = df.drop_duplicates()

### Corrige los datos si es necesario agrega una columna de fecha y hora y una columna separada para las fechas

In [ ]:
df['time'] = pd.to_datetime(df['time'], unit='s')
df['date'] = pd.to_datetime(df['time'], unit='s').dt.date
df.head()

## Estudiar y comprobar los datos

### ¿Cuántos eventos hay en los registros?

In [ ]:
print(f"Eventos totales: {len(df)}")
print(f"Conteo por tipos de evento: {df.groupby('name')['user_id'].count()}")

### ¿Cuántos usuarios y usuarias hay en los registros?

In [ ]:
print(f"Hay un total de: {df['user_id'].nunique()} usuarias y usuarios")

### ¿Cuál es el promedio de eventos por usuario?


In [ ]:
event_per_user = df.groupby('user_id')['name'].count().mean()
print(f" El promedio de eventos por usuario es de: {event_per_user}")

### ¿Qué periodo de tiempo cubren los datos? 

#### Encuentra la fecha máxima y mínima. 

In [ ]:
inicio = df['date'].min()
fin = df['date'].max()
print(f"La fecha máxima es: {fin}")
print(f"La fecha mínima es: {inicio}")

#### Traza un histograma por fecha y hora. 

In [ ]:
fig = px.histogram(df, x='time', title='Actividad por fecha y hora')

# Personalizar etiquetas
fig.update_layout(
    xaxis_title='fecha y hora',
    yaxis_title='Frecuencia',
    bargap=0.2  # Espacio entre las barras
)

fig.show()

In [ ]:
#Analizaremos unicamente por fecha
fig = px.histogram(df, x='date', title='Actividad por fecha')

# Personalizar etiquetas
fig.update_layout(
    xaxis_title='fecha',
    yaxis_title='Frecuencia',
    bargap=0.2  # Espacio entre las barras
)

fig.show()

Encontramos una muy notable diferencia a partir de agosto y en los horarios diurnos con mayor actividad.

#### ¿Puedes tener seguridad de que tienes datos igualmente completos para todo el periodo? Los eventos más antiguos podrían terminar en los registros de algunos usuarios o usuarias por razones técnicas y esto podría sesgar el panorama general. Encuentra el momento en el que los datos comienzan a estar completos e ignora la sección anterior. ¿Qué periodo representan realmente los datos?

Visualmente se observa estabilidad en los datos a partir de agosto, por lo que abarcaremos ese periodo para graficar nuevamente.

In [ ]:
#Creamos una nueva columna con el mes de eventoo
df['month'] = df['time'].dt.to_period("M").dt.start_time
#Filtramos los eventos que ocurrieron en agosto en una nueva tabla
df_ago = df[df['month'] == '2019-08-01']
df_ago.head()

In [ ]:
#Graficamos nuestra tabla filtrada por fecha y hora
fig = px.histogram(df_ago, x='time', title='Actividad por fecha y hora')

# Personalizar etiquetas
fig.update_layout(
    xaxis_title='fecha y hora',
    yaxis_title='Frecuencia',
    bargap=0.2  # Espacio entre las barras
)

fig.show()

In [ ]:
#Analizaremos unicamente por fecha
fig = px.histogram(df_ago, x='date', title='Actividad por fecha')

# Personalizar etiquetas
fig.update_layout(
    xaxis_title='fecha',
    yaxis_title='Frecuencia',
    bargap=0.2  # Espacio entre las barras
)

fig.show()

### ¿Perdiste muchos eventos y usuarios al excluir los datos más antiguos?


In [ ]:
print(f"Eventos totales: {len(df_ago)}")

In [ ]:
perdida = (1-len(df_ago)/len(df))*100
print(f"Se perdió el {round(perdida, 2)} % de los registros")

In [ ]:
u_before = df['user_id'].nunique()
u_after = df_ago['user_id'].nunique()
perdida = (1-u_after/u_before)*100
print(f"Se perdió el {round(perdida, 2)} % de los usuarios")

### Asegúrate de tener usuarios y usuarias de los tres grupos experimentales.

In [ ]:
#Hacemos un conteo de los grupos agrupando sus id con los usuarios
events = df_ago.groupby('exp_id')['user_id'].count().sort_values(ascending=False)
events

## Estudiar el embudo de eventos

### Observa qué eventos hay en los registros y su frecuencia de suceso. Ordénalos por frecuencia.

In [ ]:
#Contamos cada vez que se realizaron los eventos
print(f"Conteo por tipos de evento: {df_ago.groupby('name')['user_id'].count().sort_values(ascending=False)}")

### Encuentra la cantidad de usuarios y usuarias que realizaron cada una de estas acciones. Ordena los eventos por el número de usuarios y usuarias. 

In [ ]:
#Agrupamos para realizar el conteo considerando el numero de usuarios que realizó cada evento, el conteo es por usuarios sin repeticiión
user_by_event = df_ago.groupby('name')['user_id'].nunique().sort_values(ascending=False).reset_index()
user_by_event

In [ ]:
n_users = df_ago['user_id'].nunique()
print(f"Hay un total de: {n_users} usuarios")

#### Calcula la proporción de usuarios y usuarias que realizaron la acción al menos una vez.

In [ ]:
user_by_event['proportion'] = (user_by_event['user_id'] / n_users)*100
user_by_event

### ¿En qué orden crees que ocurrieron las acciones? ¿Todas son parte de una sola secuencia? No es necesario tenerlas en cuenta al calcular el embudo.

De acuerdo con la proporción, podemos deducir que primero tienen la visualización de la pantalla principal, posteriormente la página de las ofertas, seguido de ir al carrito, hacer el pago y ver el tutorial. Sim embargo, también sabemos que el orden no siempre es el mismo ya que no está el 100% de usuarios en la primera acción.

In [ ]:
user_by_event['users_previus_steep'] = user_by_event['user_id']. shift(1)
#columna con la proporción de conversión
user_by_event['conver_prev_steep'] = user_by_event['user_id'] / user_by_event['users_previus_steep']
#Columna con perdidas
user_by_event['dropoff_rate'] = 1 - user_by_event['conver_prev_steep']
user_by_event

### Utiliza el embudo de eventos para encontrar la proporción de usuarios y usuarias que pasan de una etapa a la siguiente. (Por ejemplo, para la secuencia de eventos A → B → C, calcula la proporción de usuarios en la etapa B a la cantidad de usuarios en la etapa A y la proporción de usuarios en la etapa C a la cantidad en la etapa B).


In [ ]:
#Usamos la función funnel, la cual genera un gráfico de embudo y permite ver las proporciones 
from plotly import graph_objects as go
fig = go.Figure(go.Funnel(
    y = user_by_event["name"],
    x = user_by_event["user_id"]
    ))
fig.show()

### ¿En qué etapa pierdes más usuarios y usuarias?

Hay más perdida de usuarios en el tutorial, seguido del segundompaso donde avanzan a la ventana offers.

### ¿Qué porcentaje de usuarios y usuarias hace todo el viaje desde su primer evento hasta el pago?

Desde el incio hasta el pago es un 46.97 %, y desde el inicio hasta el tutorial un 11.15%

## Estudiar los resultados del experimento

### ¿Cuántos usuarios y usuarias hay en cada grupo?

In [ ]:
grupos = df_ago.groupby('exp_id')['user_id'].nunique().reset_index()
grupos

### Tenemos dos grupos de control en el test A/A, donde comprobamos nuestros mecanismos y cálculos. Observa si hay una diferencia estadísticamente significativa entre las muestras 246 y 247.

In [ ]:
#Eliminamos usuarios duplicados de cada experimento
conversions = df_ago[["user_id", "exp_id"]].drop_duplicates()

#creamos un nuevo df con usuarios y un indicador de si realizó el pago o no (1 si)
converted = pd.DataFrame(data={
    "user_id": df_ago[df_ago["name"] == "PaymentScreenSuccessful"]["user_id"].unique(),
    "converted": 1
})
#unimos ambas tablas
conversions = conversions.merge(converted, on="user_id", how="left")
conversions["converted"] = conversions["converted"].fillna(0)

conversions.head(10)

In [30]:
control_1 = conversions[conversions['exp_id'] == 246]
control_2 = conversions[conversions['exp_id'] == 247]
exp = conversions[conversions['exp_id'] == 248]

In [ ]:
np.var(exp["converted"])

In [ ]:
from scipy.stats import ttest_ind

stat, p_value = ttest_ind(
    control_1["converted"],
    control_2["converted"]
)

p_value

#deteminamos alfa
alpha = 0.05
if p_value < alpha:
    print("Si hay significancia")
else:
    print("No hay significancia estadística")
    
print(f"""
Statistic: {stat}
p-value: {p_value}
""")

### Selecciona el evento más popular. 

El evento con mayor cantidad de usuarios es el primero, MainScreepAppear

#### En cada uno de los grupos de control, encuentra la cantidad de usuarios y usuarias que realizaron esta acción. 

In [ ]:
mainscreen = df[(df['name'] == 'MainScreenAppear') & (df['exp_id'].isin([246, 247]))]
mainscreen_users = mainscreen.groupby('exp_id')['user_id'].nunique().reset_index()
mainscreen_users

#### Encuentra su proporción. 

In [ ]:
mainscreen_users['proportion'] = mainscreen_users['user_id'] / n_users
mainscreen_users

In [ ]:
mainscreen.head()

#### Comprueba si la diferencia entre los grupos es estadísticamente significativa. 

In [ ]:
#Crearemos valores binarios en una nueva columna para realizar ttest
df_ago['mainscreen_flag'] = df_ago['name'].apply(lambda x: 1 if x == 'MainScreenAppear' else 0)

# Agrupar los datos por usuario para obtener un solo valor por usuario en cada grupo
group_246 = df_ago[df_ago['exp_id'] == 246].groupby('user_id')['mainscreen_flag'].max()
group_247 = df_ago[df_ago['exp_id'] == 247].groupby('user_id')['mainscreen_flag'].max()

# Realizar el t-test entre los dos grupos

import scipy.stats as stats
stat, p_value = stats.ttest_ind(group_246, group_247)

#deteminamos alfa
alpha = 0.05

if p_value < alpha:
    print("Si hay significancia")
else:
    print("No hay significancia estadística")
print(f"""
Statistic: {stat}
p-value: {p_value}
""")

#### Repite el procedimiento para todos los demás eventos (ahorrarás tiempo si creas una función especial para esta prueba). 

In [37]:
import scipy.stats as stats
#### prueba anova********** más de una prueba a la vez
def test_event_proportions_ttest(df_ago, event_name, group1, group2):
    # Crear columnas con valores binarios: 1 si el usuario hizo el evento, 0 si no
    df_ago['event_flag'] = df_ago['name'].apply(lambda x: 1 if x == event_name else 0)
    
    # Agrupar los datos por usuario para obtener un solo valor por usuario en cada grupo
    group1_data = df_ago[df_ago['exp_id'] == group1].groupby('user_id')['event_flag'].max()
    group2_data = df_ago[df_ago['exp_id'] == group2].groupby('user_id')['event_flag'].max()
    
    # Realizar el t-test entre los dos grupos
    stat, p_value = stats.ttest_ind(group1_data, group2_data)
     #deteminamos alfa ajustandolo entre el número de eventos
    alpha = 0.05 / 21
    print(f"Evento: {event_name}") 
    print(f"Statistic: {stat}")
    print(f"p-value: {p_value}")
    if p_value < alpha:
        print("Si hay significancia")
    else:
        print("No hay significancia estadística")

In [ ]:
eventos = ['MainScreenAppear', 'OffersScreenAppear', 'CartScreenAppear', 'PaymentScreenSuccessful', 'Tutorial']
for evento in eventos:
    test_event_proportions_ttest(df_ago, evento, 246, 247)

### Haz lo mismo para el grupo con fuentes alteradas. 

#### Compara los resultados con los de cada uno de los grupos de control para cada evento de forma aislada. 


In [ ]:
#Corremos nuevamente la función, ahora comparando cada grupo de control con el experimental
#Grupo 246 y 248
eventos = ['MainScreenAppear', 'OffersScreenAppear', 'CartScreenAppear', 'PaymentScreenSuccessful', 'Tutorial']
for evento in eventos:
    test_event_proportions_ttest(df_ago, evento, 246, 248)

In [ ]:
#Corremos nuevamente la función, ahora comparando cada grupo de control con el experimental
#Grupo 247 y 248
eventos = ['MainScreenAppear', 'OffersScreenAppear', 'CartScreenAppear', 'PaymentScreenSuccessful', 'Tutorial']
for evento in eventos:
    test_event_proportions_ttest(df_ago, evento, 247, 248)

#### Compara los resultados con los resultados combinados de los grupos de control.

In [ ]:
#Corremos nuevamente la función, ahora comparando los grupos de control con el experimental
eventos = ['MainScreenAppear', 'OffersScreenAppear', 'CartScreenAppear', 'PaymentScreenSuccessful', 'Tutorial']
for evento in eventos:
    test_event_proportions_ttest(df_ago, evento, 246 and 247, 248)

####  ¿Qué conclusiones puedes sacar del experimento?

De estos análisis podemos concluir los siguientes puntos:
1. Debido a que no hay significancia entre los grupos de control, podemos decir que la aleatorización es correcta, de modo que no hay sesgos entre ambos grupos y el sistema de pruebas es confiable.
2. Dado que obtenemos que no hay significancia tampoco al compararlos con el grupo b, podemos deducir que no hay pruebas de que el cambio de fuentes hayan influido en las decisiones y acciones de los usuarios.
3. Es posible que el tamaño de la muestra sea insuficiente de modo que se aconsejaría repetir la prueba con una muestra más grande.

### ¿Qué nivel de significación has establecido para probar las hipótesis estadísticas mencionadas anteriormente? 

He establecido alpha como 0.05 inicialmente, sin embargo, debido a que cada prueba se realizó 21 veces la misma función era posible acumular un falos positivo (0.05 * 21) así que para corregirlo dividí este alpha inicial entre el número de eventos y así darle esa posibilidad de falso positivo a cada evento individualmente y no acumularlo. 

#### Calcula cuántas pruebas de hipótesis estadísticas has realizado. Con un nivel de significancia estadística de 0.1, uno de cada 10 resultados podría ser falso. ¿Cuál debería ser el nivel de significación? Si deseas cambiarlo, vuelve a ejecutar los pasos anteriores y comprueba tus conclusiones.

Se realizaron 20 (+1 inicial) pruebas de hipótesis, la significancia fue de 0.05/21(n de eventos) para reducir la posibilidad de falsos positivos. Inicialmente consideré dicha corrección como 0.05/5 ya que al correr la función se ejecutaba 5 veces por prueba, sin embargo, esta corrección no consideraba los test en su totalidad.
Las hipótesis nulas se aceptan cuando se comprueba que no hubo cambios entre ambos grupos, en este caso todos los test afirman que no hay significancia estadística, dejando un margen de error muy bajo (0.00238). Por lo que no podemos rechazar las hipótesis nulas.